Vision and Cognition System 

In [2]:
import pandas as pd

for i in range(10):
    print(i**2)

0
1
4
9
16
25
36
49
64
81


In [1]:
i=0
while i<10:
    print("deepnote")
    i+=1

deepnote
deepnote
deepnote
deepnote
deepnote
deepnote
deepnote
deepnote
deepnote
deepnote


In [1]:
print('let\'s try from visual studio code')

let's try from visual studio code


In [3]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2) 

for i in range(10):
    print(fib(i))

0
1
1
2
3
5
8
13
21
34
